# Before we get started
```
conda install pandas
conda install matplotlib
```

# Why Pandas?

* No more for loops!
* Quick pictures

# How to open a csv file?

In [ ]:
ls

In [ ]:
#use the pandas data analytics library
import pandas as pd

In [ ]:
#the read_csv function loads in the file, which we assign to the df variable
df = pd.read_csv("graduation.csv")

In [ ]:
#look at the beginning of the file
df.head(3)

In [ ]:
#look at the end of the file
df.tail(3)

In [ ]:
#can also view whole, but usually impractical
df

# How do we get a quick overview?

In [ ]:
df.describe()

In [ ]:
 df.describe(include ='all')

In [ ]:
#lets look at one column
df['Demographic'].head()

In [ ]:
#if you forget the column names
df.keys()

In [ ]:
# Lets get the frequency of each demographic 
# also gives us the unique demos
df['Demographic'].value_counts()

In [ ]:
#Can also quickly figure out what sort of data is in the column
df.dtypes

# How do the columns relate to each other ?

In [ ]:
#lets look at two columns
df[['Total Cohort', 'Total Grads', 'Still Enrolled','Dropped Out']].head()

In [ ]:
#one row of data
row_100 = df[['Total Grads', 'Still Enrolled','Dropped Out']].iloc[100]
print(row_100)

In [ ]:
%matplotlib notebook
print(df[['Borough', 'Demographic']].iloc[100])
row_100.plot(kind="pie",figsize=(4, 4))


# View more than 1 row?

In [ ]:
%matplotlib inline
df.mean().plot(kind="bar")

In [ ]:
#Compare two 
df[['Total Cohort', 'Total Grads']].plot(kind='hist')

In [ ]:
#show relationship
df.plot(kind='scatter', x='Total Cohort', y='Total Grads')

Activity
=========
Compare the different types of regent grads to each other and the total cohort. 

# How can we filter this data?

In [ ]:
#filtering: lets first look at our demographics
df['Demographic'].unique()

# What mistake did we make in our graphs? 

In [ ]:
#lets just compare male and female rates instead
#http://pandas.pydata.org/pandas-docs/stable/text.html
bar_tot = df['Demographic'].str.contains('Borough Total')

In [ ]:
# lets apply that filter to our data
print(len(df[bar_tot]))
df[bar_tot].head()


In [ ]:
#hard to see, so lets visualize
df['Total Cohort'][bar_tot].plot(kind="hist")

In [ ]:
df[bar_tot].plot(kind="scatter", x="Total Cohort", y="Total Grads")

In [ ]:
# Lets find  cohorts before 2003
early = (df['Cohort']<2003)
df[early].head()

In [ ]:
#lets find female cohorts from 2003
female = df['Demographic'].str.contains('Female')
early = (df['Cohort']<2003)
fe = df[female & early]
fe.head()

In [ ]:
fe['Total Grads'].plot(kind='hist')

In [ ]:
fe.iloc[5]

Activity
========
* Find graduation rates for Asian students in cohorts with more than 5000 students 
* Find graduation rates for Hispanic students in the Bronx

# How can we use all the data together?

In [ ]:
#Can use columns just like variables
percent = df['Total Grads']/df['Total Cohort']
percent.head()

In [ ]:
#make new column
df["percent graduated"]=df['Total Grads']/df['Total Cohort']
df.head()

In [ ]:
#What about more complicated computations?
def GSI(indicators):
    cohort, grad, enrolled, dropped = indicators
    "Graduation Success Index"
    return (grad*3 - enrolled - dropped*2)/cohort
        

In [ ]:
#apply it to the data?
first = df[['Total Cohort', 'Total Grads', 'Still Enrolled', 'Total Grads']].iloc[0]
print (first)
print(GSI(first))

In [ ]:
#test if our indicator correlates to anything
%matplotlib inline
df.plot(kind="scatter", x="Total Cohort", y="GSI")

# How do we apply to the whole data set?

In [ ]:
#axis = 0 says apply to rows
#axis = 1 means apply to columns
gsi = df[['Total Cohort', 'Total Grads', 'Still Enrolled', 'Total Grads']].apply(GSI, axis=1)
gsi.head()

In [ ]:
#lets add GSI into the dataframe
df['GSI'] = gsi
df.head()

Activity
=========
* Create a regents indicator: 
    `(Advanced Regents*3+ Regents w/o*2 + Local) / Total Regents`
* Create a function that computes a regents indicator 
* Apply the function to the dataframe 
* Create a new column RSI that stores the index for each row

# BREAK

# How can we start summarizing data?

In [ ]:
#groupby is for one function
demo = df.groupby("Borough")

In [ ]:
#can use any function
demo.mean()

In [ ]:
demo.std()

In [ ]:
#can group by two catagories:
bt = df.groupby(["Borough", "Demographic"]).mean()
bt

In [ ]:
#sometimes looks better flipped
bt.T

In [ ]:
#can filter on cols
bt[["Total Cohort","GSI", "percent graduated"]].T

In [ ]:
#Can do groupby on filtered data
male = df['Demographic'].str.contains("Male")
female = df['Demographic'].str.contains("Female")
binary = df[(male | female)].groupby(["Borough", "Demographic"]).mean()
binary

In [ ]:
#filter along table (Indexing)
binary["Total Grads"]["Bronx"].plot(kind='Bar')

In [ ]:
#can groupby indefinite number of levels
many = df[(male | female)].groupby(["Borough", "Demographic", "Cohort"]).mean()
many

# what if we want more than 1 function?

In [ ]:
# total number of items but the last start date
"""index becomes the row agg function is how 
data in each subset->(Station, Date) is analyzed:
* first argument is the column being analyzed
* second is aggregating function:
    * total items
    * max start date
"""
#now fancy grouping
df_pivot = df[(male|female)].pivot_table(index='Cohort',
                          columns = "Demographic",
            aggfunc={'Total Cohort':'median',
                     'Total Grads':'median',
                     'Still Enrolled':'median',
                     'Dropped Out':'median',
                     'percent graduated': 'std'})
df_pivot

In [ ]:
df_pivot[['Still Enrolled', 'Dropped Out', 'Total Grads']].plot(figsize=(15,5))

In [ ]:
#Any other cat grouping

df['Demographic'].unique()
asian = df['Demographic'].str.contains("Asian")
black = df['Demographic'].str.contains("Black")
hispanic = df['Demographic'].str.contains("Hispanic")
white = df['Demographic'].str.contains("White")
ind_cols = ["Total Cohort","Total Grads", "Still Enrolled", ""]
from pandas.tools.plotting import parallel_coordinates
parallel_coordinates(df[asian|black|hispanic|white], 'Demographic', cols=["Total "])

In [ ]:
parallel_coordinates?